In [2]:
from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import Input,Dense,Reshape,Flatten
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
import numpy as np
import random
import tensorflow as tf

In [3]:
img_rows = 28
img_cols = 28
channels = 1
img_shape = (img_rows,img_cols,channels)

In [4]:
def build_generator():
    noise_shape = (100,)
    model = Sequential()
    model.add(Dense(256,input_shape=noise_shape))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(512))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(1024))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    
    model.add(Dense(np.prod(img_shape),activation='tanh'))
    model.add(Reshape(img_shape))
    
    model.summary()
    
    noise = Input(shape=noise_shape)
    img = model(noise)
    return Model(noise,img)

In [5]:
def build_descriminator():
    model= Sequential()
    model.add(Flatten(input_shape=img_shape))
    model.add(Dense(512))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(256))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(1,activation='sigmoid'))
    model.summary()
    
    img=Input(shape=img_shape)
    validity=model(img)
    return Model(img,validity)

In [6]:
 def train(epochs,batch_size=128,save_interval=50):
        (x_train,_),(_,_)=mnist.load_data()
        x_train = (x_train.astype(np.float32)-127.5)/127.5
        x_train = np.expand_dims(x_train,axis=3)
        half_batch = int(batch_size/2)
        for epoch in range(epochs):
            idx = np.random.randint(0,x_train.shape[0],half_batch)
            imgs = x_train[idx]
            noise = np.random.normal(0,1,(half_batch,100)) 
            gen_imgs = generator.predict(noise)
            d_loss_real = discriminator.train_on_batch(imgs,np.ones((half_batch,1)))
            d_loss_fake = discriminator.train_on_batch(gen_imgs,np.zeros((half_batch,1)))
            
            d_loss = 0.5 * np.add(d_loss_real,d_loss_fake)
            
            noise = np.random.normal(0,1,(batch_size,100))
            valid_y = np.array([1]*batch_size)
            g_loss = combined.train_on_batch(noise,valid_y)
            
            print("%d [D Loss: %f, Acc.: %0.2f%%] [G Loss: %f]"% (epoch,d_loss[0],100*d_loss[1],g_loss))
            
            if(epoch % save_interval == 0 ):
                save_imgs(epoch)
                

In [18]:
def save_imgs(epoch):
    r,c = 5,5
    noise = np.random.normal(0,1,(r*c,100))
    gen_imgs = generator.predict(noise)
    gen_imgs = 0.5 * gen_imgs + 0.5
    fig,axs = plt.subplots(r,c)
    cnt = 0
    for i in range(r):
        for j in range(c):
            axs[i,j].imshow(gen_imgs[cnt,:,:,0],cmap='gray')
            axs[i,j].axis('off')
            cnt+=1
    fig.savefig("D:\Dataset\Test\mnist_%d.png"%epoch)
    plt.close()

In [8]:
def define_gan(generator, discriminator):
	# make weights in the discriminator not trainable
	discriminator.trainable = False
	# connect them
	model = Sequential()
	# add generator
	model.add(generator)
	# add the discriminator
	model.add(discriminator)
	# compile model
	opt = Adam(lr=0.0002, beta_1=0.5)
	model.compile(loss='binary_crossentropy', optimizer=opt)
	return model

In [9]:
optimizer = Adam(0.0002,0.5)
discriminator = build_descriminator()
discriminator.compile(loss='binary_crossentropy',
                      optimizer=optimizer,
                     metrics=['accuracy'])

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense (Dense)                (None, 512)               401920    
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               131328    
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 257       
Total params: 533,505
Trainable params: 533,505
Non-trainable params: 0
__________________________________________________

In [13]:
generator = build_generator()
generator.compile(loss='binary_crossentropy',optimizer=optimizer)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 256)               25856     
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 256)               0         
_________________________________________________________________
batch_normalization (BatchNo (None, 256)               1024      
_________________________________________________________________
dense_4 (Dense)              (None, 512)               131584    
_________________________________________________________________
leaky_re_lu_3 (LeakyReLU)    (None, 512)               0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 512)               2048      
_________________________________________________________________
dense_5 (Dense)              (None, 1024)             

In [14]:
z=Input(shape=(100,))

In [15]:
img = generator(z)


In [16]:
combined = define_gan(generator,discriminator)

In [11]:
discriminator.trainable = False

In [38]:
valid = discriminator(img)

In [39]:
combined = Model(z,valid)
combined.compile(loss='binary_crossentropy',optimizer=optimizer)

In [19]:
train(epochs=100000,batch_size=32,save_interval=200)

0 [D Loss: 0.396719, Acc.: 71.88%] [G Loss: 0.669676]
1 [D Loss: 0.365033, Acc.: 68.75%] [G Loss: 0.732362]
2 [D Loss: 0.341151, Acc.: 81.25%] [G Loss: 0.787190]
3 [D Loss: 0.338431, Acc.: 84.38%] [G Loss: 0.836685]
4 [D Loss: 0.306261, Acc.: 81.25%] [G Loss: 0.969622]
5 [D Loss: 0.271458, Acc.: 96.88%] [G Loss: 1.121125]
6 [D Loss: 0.228697, Acc.: 100.00%] [G Loss: 1.232662]
7 [D Loss: 0.218446, Acc.: 100.00%] [G Loss: 1.324153]
8 [D Loss: 0.159859, Acc.: 100.00%] [G Loss: 1.573921]
9 [D Loss: 0.152613, Acc.: 100.00%] [G Loss: 1.631724]
10 [D Loss: 0.118188, Acc.: 100.00%] [G Loss: 1.756233]
11 [D Loss: 0.109221, Acc.: 100.00%] [G Loss: 1.922555]
12 [D Loss: 0.097611, Acc.: 100.00%] [G Loss: 2.006845]
13 [D Loss: 0.107604, Acc.: 100.00%] [G Loss: 2.130295]
14 [D Loss: 0.075587, Acc.: 100.00%] [G Loss: 2.250367]
15 [D Loss: 0.075966, Acc.: 100.00%] [G Loss: 2.330700]
16 [D Loss: 0.056928, Acc.: 100.00%] [G Loss: 2.404482]
17 [D Loss: 0.054977, Acc.: 100.00%] [G Loss: 2.444279]
18 [D Lo

KeyboardInterrupt: 

In [51]:
noise = Input(shape=(28,28,1))
noise.shape

TensorShape([None, 28, 28, 1])

In [52]:
(x_train,_),(_,_)=mnist.load_data()

In [54]:
x_train = np.expand_dims(x_train,axis=3)

In [58]:
x_train[0].shape

(28, 28, 1)

In [59]:
idx = np.random.randint(0,x_train.shape[0],64)

In [63]:
noise = np.random.normal(0, 1, (64, 100))

In [70]:
np.zeros((64,1 ))

array([[0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.]])

# For Generating image using the Trained Model

In [ ]:
model = load_model('generator_model.h5')

In [ ]:
vector=randn(100)
vector=vector.reshape(1,100)
X = model.predict(vector)
pyplot.imshow(X[0,:,:,0],cmap='gray_r')
pyplot.show()